# Pipeline

## Functions

In [ ]:
# Imports
import os
import cv2
import pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from noise_filter import preprocess_image
import background_remover_w2 as background_remover
from image_split_team5 import segment_multiple_paintings

# =========================
# Matchers & Evaluation
# =========================

def create_matcher(descriptor_type: str):
    """
    Return an OpenCV matcher appropriate for the descriptor type.
    - SIFT/CSIFT: FLANN (KD-tree, L2)
    - ORB:        BFMatcher with Hamming distance
    """
    descriptor_type = descriptor_type.upper()
    if descriptor_type in ("SIFT", "CSIFT"):
        return cv2.FlannBasedMatcher(dict(algorithm=1, trees=8), dict(checks=64))  # KDTree
    elif descriptor_type == "ORB":
        return cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    else:
        raise ValueError(f"Unsupported descriptor_type: {descriptor_type}")


def good_match_count(desQ, desD, matcher, ratio=0.76):
    """
    2-NN + Lowe ratio; returns number of 'good' matches.
    Works for both float (SIFT/CSIFT) and binary (ORB) descriptors.
    """
    if desQ is None or desD is None or len(desQ) == 0 or len(desD) == 0:
        return 0
    knn = matcher.knnMatch(desQ, desD, k=2)
    return sum(1 for m, n in knn if m.distance < ratio * n.distance)


def rank_db_for_query_multi(desc_crops, db_descs, matcher, ratio=0.76):
    scores = []
    for i, desD in enumerate(db_descs):
        best = max(good_match_count(desQ, desD, matcher, ratio) for desQ in desc_crops)
        scores.append((i, best))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores


def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]
    score, num_hits = 0.0, 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)


def mapk(actual, predicted, k=10, skip_unknown=True):
    map_score, total_pics = 0.0, 0
    for idx, p in enumerate(predicted):
        if len(p) == 2:
            apk_score1 = apk([actual[idx][0]], p[0], k)
            apk_score2 = apk([actual[idx][1]], p[1], k)
            map_score += apk_score1 + apk_score2
            total_pics += 2
        elif actual[idx] == [-1] and skip_unknown:
            continue
        else:
            apk_score = apk(actual[idx], p, k)
            map_score += apk_score
            total_pics += 1
    return map_score / total_pics if total_pics > 0 else 0.0


def compute_matches(desc_query, db_descs, gt_corresps, matcher, ratio=0.76, rate_threshold=0.125, descriptor_type=None):
    """
    desc_query:  list where each element is [desc_single] or [desc_left, desc_right]
    db_descs:    list of descriptor arrays, one per DB image
    gt_corresps: list; each entry is an int, list of ints, or [-1]
    """
    predicted = []
    min_score_rate_correct = float('inf')
    max_score_rate_incorrect = float('-inf')

    for qi, desc_crops in tqdm(enumerate(desc_query), total=len(desc_query), desc="Processing queries"):
        q_gt = gt_corresps[qi]
        if len(q_gt) == 2 and len(desc_crops) == 2:
            list_ranked_indices = []
            for desc_crop in desc_crops:
                ranked = rank_db_for_query_multi([desc_crop], db_descs, matcher, ratio=ratio)
                ranked_indices, ranked_scores = zip(*ranked)
                rate = ranked_scores[0] / desc_crop.shape[0] if desc_crop is not None and desc_crop.shape[0] > 0 else 0
                if rate < rate_threshold:
                    ranked_indices = [-1]
                list_ranked_indices.append(ranked_indices)

                if ranked_indices[0] in q_gt:
                    min_score_rate_correct = min(min_score_rate_correct, rate)
                else:
                    max_score_rate_incorrect = max(max_score_rate_incorrect, rate)

                # print the rate, gt and predicted for each crop
                # print(f"Query {qi} GT: {q_gt}, Predicted: {ranked_indices[0]}, Rate: {rate:.4f}")

        else:
            ranked = rank_db_for_query_multi(desc_crops, db_descs, matcher, ratio=ratio)
            list_ranked_indices, list_ranked_scores = zip(*ranked)
            base = desc_crops[0] if (len(desc_crops) > 0 and desc_crops[0] is not None) else None
            rate = (list_ranked_scores[0] / base.shape[0]) if (base is not None and base.shape[0] > 0) else 0
            if rate < rate_threshold:
                list_ranked_indices = [-1]
            
            if list_ranked_indices[0] in q_gt:
                min_score_rate_correct = min(min_score_rate_correct, rate)
            else:
                max_score_rate_incorrect = max(max_score_rate_incorrect, rate)

            # print the rate, gt and predicted for the query
            # print(f"Query {qi} GT: {q_gt}, Predicted: {list_ranked_indices[0]}, Rate: {rate:.4f}")

        predicted.append(list_ranked_indices)

    print(f"Min score for correct matches: {min_score_rate_correct}")
    print(f"Max score for incorrect matches: {max_score_rate_incorrect}")

    # save the predicted results to a file
    if descriptor_type:
        with open(f"results/predicted_results_{descriptor_type}_rate{rate_threshold:.3f}.pkl", "wb") as f:
            pickle.dump(predicted, f)
    return predicted


# =========================
# I/O & Preprocessing
# =========================

def load_images_from_folder(folder, exts=(".jpg",)):
    names = sorted([f for f in os.listdir(folder) if f.lower().endswith(exts)])
    imgs = []
    for name in names:
        path = os.path.join(folder, name)
        img = cv2.imread(path)
        if img is None:
            print(f"⚠️ Could not read {path}")
            continue
        imgs.append(img)
    return names, imgs


def validate_split(is_split, imgs, min_size_ratio=0.2):
    if not is_split:
        return False, imgs
    if not isinstance(imgs, (list, tuple)) or len(imgs) != 2:
        return False, imgs

    img1, img2 = imgs
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    total_width = w1 + w2
    total_height = max(h1, h2)

    valid_left   = (w1 / total_width)  > min_size_ratio
    valid_right  = (w2 / total_width)  > min_size_ratio
    valid_h1     = (h1 / total_height) > min_size_ratio
    valid_h2     = (h2 / total_height) > min_size_ratio

    valid_imgs = []
    if valid_left and valid_h1:
        valid_imgs.append(img1)
    if valid_right and valid_h2:
        valid_imgs.append(img2)

    if len(valid_imgs) == 2:
        return True, valid_imgs
    elif len(valid_imgs) == 1:
        return False, valid_imgs[0]
    else:
        return False, imgs


# =========================
# Descriptors (SIFT / ORB / Color-SIFT)
# =========================

def _to_uint8(img):
    """Normalize/convert single-channel float image to uint8 safely."""
    if img.dtype == np.uint8:
        return img
    m, M = np.min(img), np.max(img)
    if M <= m + 1e-12:
        return np.zeros_like(img, dtype=np.uint8)
    out = (255.0 * (img - m) / (M - m)).astype(np.uint8)
    return out

def compute_sift_desc(
    img_bgr,
    nfeatures=1200,
    rootsift=False,
    img_name="",
    show_img=False,
    save_vis=False,
    vis_dir="results/query_keypoints",
    info=False,
):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(nfeatures=nfeatures)
    kp, des = sift.detectAndCompute(gray, None)
    if des is None:
        if info:
            print(f"⚠️ No SIFT descriptors for {img_name}.")
        return None

    des = des.astype(np.float32)
    if rootsift:
        des /= (des.sum(axis=1, keepdims=True) + 1e-12)
        des = np.sqrt(des, out=des)

    if show_img or save_vis:
        vis = cv2.drawKeypoints(gray, kp, img_bgr.copy(), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        if save_vis:
            os.makedirs(vis_dir, exist_ok=True)
            cv2.imwrite(os.path.join(vis_dir, f"{img_name}_keypoints.jpg"), vis)
        if show_img:
            plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
            plt.title(f"SIFT Keypoints - {img_name}")
            plt.axis('off')
            plt.show()

    if info:
        print(f"SIFT descriptors for {img_name}: {des.shape[0]} keypoints.")
    return des


def compute_orb_desc(
    img_bgr,
    nfeatures=2000,
    fast_threshold=20,
    score_type="HARRIS",   # or "FAST"
    img_name="",
    show_img=False,
    save_vis=False,
    vis_dir="results/query_keypoints",
    info=False,
):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    score = cv2.ORB_HARRIS_SCORE if score_type.upper() == "HARRIS" else cv2.ORB_FAST_SCORE
    orb = cv2.ORB_create(
        nfeatures=nfeatures,
        scaleFactor=1.2,
        nlevels=8,
        edgeThreshold=31,
        firstLevel=0,
        WTA_K=2,
        scoreType=score,
        patchSize=31,
        fastThreshold=fast_threshold
    )
    kp, des = orb.detectAndCompute(gray, None)
    if des is None:
        if info:
            print(f"⚠️ No ORB descriptors for {img_name}.")
        return None

    if show_img or save_vis:
        vis = cv2.drawKeypoints(gray, kp, img_bgr.copy(), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        if save_vis:
            os.makedirs(vis_dir, exist_ok=True)
            cv2.imwrite(os.path.join(vis_dir, f"{img_name}_keypoints.jpg"), vis)
        if show_img:
            plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
            plt.title(f"ORB Keypoints - {img_name}")
            plt.axis('off')
            plt.show()

    if info:
        print(f"ORB descriptors for {img_name}: {des.shape[0]} keypoints.")
    return des


def compute_color_sift_desc(
    img_bgr,
    nfeatures=1200,
    color_space="OPPONENT",   # "OPPONENT" | "RGB" | "HSV"
    rootsift=False,
    img_name="",
    show_img=False,
    save_vis=False,
    vis_dir="results/query_keypoints",
    info=False,
):
    """
    Color-SIFT: detect keypoints once (intensity), then compute SIFT on 3 channels
    and concatenate per-keypoint (3 * 128 = 384 dims). Float32, L2.
    """
    # 1) Keypoints on intensity (grayscale or opponent O3)
    bgr = img_bgr
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(nfeatures=nfeatures)
    kp = sift.detect(gray, None)
    if kp is None or len(kp) == 0:
        if info:
            print(f"⚠️ No keypoints for Color-SIFT: {img_name}.")
        return None

    # 2) Build 3 channels per chosen space as uint8 single-channel images
    color_space = color_space.upper()
    if color_space == "RGB":
        r, g, b = cv2.split(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB))
        chs = [r, g, b]
    elif color_space == "HSV":
        h, s, v = cv2.split(cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV))
        chs = [h, s, v]
    else:
        # Opponent color space (van de Sande et al. 2010)
        b, g, r = cv2.split(bgr.astype(np.float32))
        o1 = (r - g) / np.sqrt(2.0)
        o2 = (r + g - 2.0 * b) / np.sqrt(6.0)
        o3 = (r + g + b) / np.sqrt(3.0)
        chs = [_to_uint8(o1), _to_uint8(o2), _to_uint8(o3)]

    # 3) Compute SIFT on each channel using the SAME keypoints; then concat
    desc_list = []
    for ch in chs:
        _kp, des_ch = sift.compute(ch, kp)  # use same keypoints
        if des_ch is None:
            if info:
                print(f"⚠️ Channel produced no descriptors for {img_name}.")
            return None
        desc_list.append(des_ch.astype(np.float32))

    # Ensure same #rows for all channels
    min_rows = min(d.shape[0] for d in desc_list)
    if min_rows == 0:
        if info:
            print(f"⚠️ No descriptors after channel alignment for {img_name}.")
        return None
    desc_list = [d[:min_rows] for d in desc_list]
    des = np.concatenate(desc_list, axis=1)  # (N, 384)

    # 4) RootSIFT (on concatenated descriptor)
    if rootsift:
        des /= (des.sum(axis=1, keepdims=True) + 1e-12)
        des = np.sqrt(des, out=des)

    # (Optional) visualization using original gray & kp
    if show_img or save_vis:
        vis = cv2.drawKeypoints(gray, kp[:min_rows], bgr.copy(), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        if save_vis:
            os.makedirs(vis_dir, exist_ok=True)
            cv2.imwrite(os.path.join(vis_dir, f"{img_name}_keypoints.jpg"), vis)
        if show_img:
            plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
            plt.title(f"Color-SIFT Keypoints - {img_name} ({color_space})")
            plt.axis('off')
            plt.show()

    if info:
        print(f"Color-SIFT ({color_space}) for {img_name}: {des.shape[0]} keypoints, {des.shape[1]} dims.")
    return des


# =========================
# Descriptor Pipelines
# =========================

def build_or_load_db_descriptors(
    db_folder,
    desc_db_path,
    descriptor_type="SIFT",
    nfeatures=1200,
    rootsift=True,
    color_space="OPPONENT",
    orb_fast_threshold=20,
    orb_score_type="HARRIS",
    use_cache=True,
    progress=True,
):
    if use_cache and os.path.exists(desc_db_path):
        print(f"✅ Loading DB descriptors from {desc_db_path}")
        with open(desc_db_path, "rb") as f:
            data = pickle.load(f)
            return data["desc_gt"], data["gt_names"]

    print(f"🧠 Computing {descriptor_type} descriptors for database images...")
    db_names, db_imgs = load_images_from_folder(db_folder)
    db_descs = []
    iterator = tqdm(zip(db_imgs, db_names), total=len(db_names)) if progress else zip(db_imgs, db_names)

    descriptor_type = descriptor_type.upper()
    for img, name in iterator:
        stem = os.path.splitext(name)[0]
        if descriptor_type == "SIFT":
            d = compute_sift_desc(img, nfeatures=nfeatures, rootsift=rootsift, img_name=stem)
        elif descriptor_type == "ORB":
            d = compute_orb_desc(img, nfeatures=nfeatures, fast_threshold=orb_fast_threshold,
                                 score_type=orb_score_type, img_name=stem)
        elif descriptor_type == "CSIFT":
            d = compute_color_sift_desc(img, nfeatures=nfeatures, color_space=color_space,
                                        rootsift=rootsift, img_name=stem)
        else:
            raise ValueError(f"Unsupported descriptor_type: {descriptor_type}")
        db_descs.append(d)

    with open(desc_db_path, "wb") as f:
        pickle.dump({"desc_gt": db_descs, "gt_names": db_names}, f)
    print(f"✅ Saved DB descriptors to {desc_db_path}")
    return db_descs, db_names


def single_img_removal(
    img,
    img_name,
    background_remover,
    descriptor_type="SIFT",
    nfeatures=1200,
    rootsift=True,
    color_space="OPPONENT",
    orb_fast_threshold=20,
    orb_score_type="HARRIS",
    bg_crop_min_ratio=0.10,
    show_img=False,
    info=False,
):
    im, mask, *_ = background_remover.remove_background_morphological_gradient(img)
    cropped = background_remover.crop_to_mask_rectangle(img, mask)

    if (cropped.shape[1] / img.shape[1] < bg_crop_min_ratio) or (cropped.shape[0] / img.shape[0] < bg_crop_min_ratio):
        cropped = img

    stem = os.path.splitext(img_name)[0] + "_single"
    descriptor_type = descriptor_type.upper()
    if descriptor_type == "SIFT":
        return compute_sift_desc(cropped, nfeatures=nfeatures, rootsift=rootsift,
                                 img_name=stem, show_img=show_img, save_vis=show_img, info=info)
    elif descriptor_type == "ORB":
        return compute_orb_desc(cropped, nfeatures=nfeatures, fast_threshold=orb_fast_threshold,
                                score_type=orb_score_type, img_name=stem,
                                show_img=show_img, save_vis=show_img, info=info)
    elif descriptor_type == "CSIFT":
        return compute_color_sift_desc(cropped, nfeatures=nfeatures, color_space=color_space,
                                       rootsift=rootsift, img_name=stem,
                                       show_img=show_img, save_vis=show_img, info=info)
    else:
        raise ValueError(f"Unsupported descriptor_type: {descriptor_type}")


def compute_query_descriptors(
    q_imgs,
    q_names,
    preprocess_image,
    segment_multiple_paintings,
    validate_split_fn=validate_split,
    background_remover=None,
    descriptor_type="SIFT",
    nfeatures=1200,
    rootsift=True,
    color_space="OPPONENT",
    orb_fast_threshold=20,
    orb_score_type="HARRIS",
    crop_min_ratio_single=0.20,
    bg_crop_min_ratio=0.10,
    show_img=False,
    info=False,
):
    desc_query = []
    descriptor_type = descriptor_type.upper()

    for img, img_name in tqdm(zip(q_imgs, q_names), total=len(q_names), desc="Processing queries"):
        original_h, original_w = img.shape[:2]
        img_proc = preprocess_image(img)
        is_split, split_imgs = segment_multiple_paintings(img_proc)
        ok_split, parts = validate_split_fn(is_split, split_imgs)

        if ok_split:
            left_artwork, right_artwork = parts

            if background_remover is not None:
                iml, left_mask, *_  = background_remover.remove_background_morphological_gradient(left_artwork)
                imr, right_mask, *_ = background_remover.remove_background_morphological_gradient(right_artwork)
                left_cropped  = background_remover.crop_to_mask_rectangle(left_artwork, left_mask)
                right_cropped = background_remover.crop_to_mask_rectangle(right_artwork, right_mask)
            else:
                left_cropped, right_cropped = left_artwork, right_artwork

            too_small = (
                (left_cropped.shape[1]  / original_w < crop_min_ratio_single) or
                (left_cropped.shape[0]  / original_h < crop_min_ratio_single) or
                (right_cropped.shape[1] / original_w < crop_min_ratio_single) or
                (right_cropped.shape[0] / original_h < crop_min_ratio_single)
            )

            if too_small:
                if info:
                    print("Processing image", img_name, "as single due to small crop size")
                desc_single = single_img_removal(
                    img_proc, img_name, background_remover,
                    descriptor_type=descriptor_type, nfeatures=nfeatures, rootsift=rootsift, 
                    bg_crop_min_ratio=bg_crop_min_ratio, color_space=color_space,
                    orb_fast_threshold=orb_fast_threshold, orb_score_type=orb_score_type,
                    show_img=show_img, info=info
                )
                desc_query.append([desc_single])
            else:
                stem = os.path.splitext(img_name)[0]
                if descriptor_type == "SIFT":
                    desc_left = compute_sift_desc(
                        left_cropped, nfeatures=nfeatures, rootsift=rootsift,
                        img_name=stem + "_left", show_img=show_img, save_vis=show_img, info=info
                    )
                    desc_right = compute_sift_desc(
                        right_cropped, nfeatures=nfeatures, rootsift=rootsift,
                        img_name=stem + "_right", show_img=show_img, save_vis=show_img, info=info
                    )
                elif descriptor_type == "ORB":
                    desc_left = compute_orb_desc(
                        left_cropped, nfeatures=nfeatures, fast_threshold=orb_fast_threshold,
                        score_type=orb_score_type, img_name=stem + "_left",
                        show_img=show_img, save_vis=show_img, info=info
                    )
                    desc_right = compute_orb_desc(
                        right_cropped, nfeatures=nfeatures, fast_threshold=orb_fast_threshold,
                        score_type=orb_score_type, img_name=stem + "_right",
                        show_img=show_img, save_vis=show_img, info=info
                    )
                elif descriptor_type == "CSIFT":
                    desc_left = compute_color_sift_desc(
                        left_cropped, nfeatures=nfeatures, color_space=color_space,
                        rootsift=rootsift, img_name=stem + "_left",
                        show_img=show_img, save_vis=show_img, info=info
                    )
                    desc_right = compute_color_sift_desc(
                        right_cropped, nfeatures=nfeatures, color_space=color_space,
                        rootsift=rootsift, img_name=stem + "_right",
                        show_img=show_img, save_vis=show_img, info=info
                    )
                else:
                    raise ValueError(f"Unsupported descriptor_type: {descriptor_type}")
                desc_query.append([desc_left, desc_right])
        else:
            desc_single = single_img_removal(
                img_proc, img_name, background_remover,
                descriptor_type=descriptor_type, nfeatures=nfeatures, rootsift=rootsift, 
                bg_crop_min_ratio=bg_crop_min_ratio, color_space=color_space,
                orb_fast_threshold=orb_fast_threshold, orb_score_type=orb_score_type,
                show_img=show_img, info=info
            )
            desc_query.append([desc_single])
    return desc_query


## SIFT

In [4]:
# =========================
# Main (example wiring)
# =========================

# --- Config (adjust as needed) ---
DB_FOLDER      = "../Data/BBDD/"
QUERY_FOLDER   = "../Data/Week4/qsd1_w4/"
GT_PATH        = "../Data/Week4/qsd1_w4/gt_corresps.pkl"
DESC_DB_PATH   = "results/descriptors_db_sift.pkl"

DESCRIPTOR         = "SIFT"

# SIFT & matching parameters
NFEATURES          = 1200
ROOTSIFT           = True
LOWE_RATIO         = 0.76       # Lowe ratio for 2-NN filtering
RATE_THRESHOLD     = 0.125      # good_matches/num_desc threshold to mark unknown
MIN_SIZE_RATIO     = 0.20       # split validation
CROP_MIN_RATIO     = 0.20       # single fallback if either crop too small
BG_CROP_MIN_RATIO  = 0.10       # background crop too small -> use original
SHOW_VIS           = False
INFO_PRINT         = False

os.makedirs("results", exist_ok=True)

# 1) Matcher
matcher = create_matcher(DESCRIPTOR)

# 2) DB descriptors
db_descs, db_names = build_or_load_db_descriptors(
    DB_FOLDER,
    DESC_DB_PATH,
    nfeatures=NFEATURES,
    rootsift=ROOTSIFT,
    use_cache=True,
    progress=True,
)

# 3) Queries + GT
print("📥 Loading query images...")
q_names, q_imgs = load_images_from_folder(QUERY_FOLDER)
print("📥 Loading ground-truth correspondences...")
with open(GT_PATH, "rb") as f:
    gt_corresps = pickle.load(f)
print(f"→ {len(gt_corresps)} GT entries loaded")

# 3) Query descriptors
print("🧠 Computing SIFT for queries...")
desc_query_sift = compute_query_descriptors(
    q_imgs,
    q_names,
    preprocess_image=preprocess_image,
    segment_multiple_paintings=segment_multiple_paintings,
    validate_split_fn=lambda is_split, imgs: validate_split(is_split, imgs, min_size_ratio=MIN_SIZE_RATIO),
    background_remover=background_remover,
    nfeatures=NFEATURES,
    rootsift=ROOTSIFT,
    crop_min_ratio_single=CROP_MIN_RATIO,
    bg_crop_min_ratio=BG_CROP_MIN_RATIO,
    show_img=SHOW_VIS,
    info=INFO_PRINT,
)

# 4) Ranking + mAP
print("🏆 Evaluating retrieval performance...")
pred_sift = compute_matches(
    desc_query_sift,
    db_descs,
    gt_corresps,
    matcher=matcher,
    ratio=LOWE_RATIO,
    rate_threshold=RATE_THRESHOLD
)
map5_sift = mapk(gt_corresps, pred_sift, k=5, skip_unknown=False)
map1_sift = mapk(gt_corresps, pred_sift, k=1, skip_unknown=False)

print(f"✅ mAP@1 = {map1_sift:.4f}")
print(f"✅ mAP@5 = {map5_sift:.4f}")


✅ Loading DB descriptors from results/descriptors_db_sift.pkl
📥 Loading query images...
📥 Loading ground-truth correspondences...
→ 30 GT entries loaded
🏆 Evaluating retrieval performance...


Processing queries: 100%|██████████| 30/30 [04:15<00:00,  8.53s/it]

Min score for correct matches: 0.013333333333333334
Max score for incorrect matches: 0.2722731057452123
✅ mAP@1 = 0.8205
✅ mAP@5 = 0.8205


In [5]:
#print pred
for i in range(len(pred_sift)):
    if len(pred_sift[i]) == 2:
        print(f"Query image: {q_names[i]} - GT: {gt_corresps[i]} - Predicted DB indices left: {pred_sift[i][0][:5]}, right: {pred_sift[i][1][:5]}")
    else:
        print(f"Query image: {q_names[i]} - GT: {gt_corresps[i]} - Predicted DB indices: {pred_sift[i][:5]}")

Query image: 00000.jpg - GT: [-1] - Predicted DB indices: [-1]
Query image: 00001.jpg - GT: [150] - Predicted DB indices: (150, 143, 95, 204, 255)
Query image: 00002.jpg - GT: [48, 251] - Predicted DB indices left: (48, 88, 239, 67, 70), right: (251, 210, 90, 51, 105)
Query image: 00003.jpg - GT: [32] - Predicted DB indices: (32, 165, 105, 118, 66)
Query image: 00004.jpg - GT: [161] - Predicted DB indices: (161, 157, 236, 271, 30)
Query image: 00005.jpg - GT: [81] - Predicted DB indices: (81, 99, 268, 284, 143)
Query image: 00006.jpg - GT: [62, 38] - Predicted DB indices left: (62, 238, 27, 53, 72), right: (38, 105, 88, 101, 209)
Query image: 00007.jpg - GT: [-1] - Predicted DB indices: (66, 238, 137, 236, 157)
Query image: 00008.jpg - GT: [128] - Predicted DB indices: [-1]
Query image: 00009.jpg - GT: [155, 258] - Predicted DB indices left: (155, 199, 2, 204, 150), right: (258, 128, 199, 220, 112)
Query image: 00010.jpg - GT: [136, 76] - Predicted DB indices left: (136, 105, 211, 238,

## ORB

In [ ]:
DB_FOLDER      = "../Data/BBDD/"
QUERY_FOLDER   = "../Data/Week4/qsd1_w4/"
GT_PATH        = "../Data/Week4/qsd1_w4/gt_corresps.pkl"

DESCRIPTOR         = "ORB"       # <- switch between "SIFT" and "ORB"
NFEATURES          = 2000        # both SIFT & ORB accept this parameter- For SIFT, typical is 1200; for ORB, typical is 2000
ROOTSIFT           = False       # ignored for ORB
ORB_FAST_THRESHOLD = 20
ORB_SCORE_TYPE     = "HARRIS"    # or "FAST"

LOWE_RATIO         = 0.76
RATE_THRESHOLD     = 0.06
MIN_SIZE_RATIO     = 0.20
CROP_MIN_RATIO     = 0.20
BG_CROP_MIN_RATIO  = 0.10
SHOW_VIS           = False
INFO_PRINT         = False

# Rate thereshold = 0.125 gives:
# ✅ mAP@1 = 0.7692
# ✅ mAP@5 = 0.7692
# Rate threshold = 0.09 gives:
# ✅ mAP@1 = 0.8205
# ✅ mAP@5 = 0.8205
# Rate thereshold = 0.015 gives:
# ✅ mAP@1 = 0.7692
# ✅ mAP@5 = 0.7842
# Rate thereshold = 0.06 gives:
# ✅ mAP@1 = 0.8718
# ✅ mAP@5 = 0.8718

# Cache file name reflects descriptor
DESC_DB_PATH   = f"results/descriptors_db_{DESCRIPTOR.lower()}.pkl"

os.makedirs("results", exist_ok=True)

# 1) Build matcher
matcher = create_matcher(DESCRIPTOR)

# 2) DB descriptors
db_descs, db_names = build_or_load_db_descriptors(
    DB_FOLDER,
    DESC_DB_PATH,
    descriptor_type=DESCRIPTOR,
    nfeatures=NFEATURES,
    rootsift=ROOTSIFT,
    orb_fast_threshold=ORB_FAST_THRESHOLD,
    orb_score_type=ORB_SCORE_TYPE,
    use_cache=True,
    progress=True,
)

# 3) Queries + GT
print("📥 Loading query images...")
q_names, q_imgs = load_images_from_folder(QUERY_FOLDER)
print("📥 Loading ground-truth correspondences...")
with open(GT_PATH, "rb") as f:
    gt_corresps = pickle.load(f)
print(f"→ {len(gt_corresps)} GT entries loaded")

# 4) Query descriptors
print(f"🧠 Computing {DESCRIPTOR} for queries...")
desc_query_orb = compute_query_descriptors(
    q_imgs,
    q_names,
    preprocess_image=preprocess_image,
    segment_multiple_paintings=segment_multiple_paintings,
    validate_split_fn=lambda is_split, imgs: validate_split(is_split, imgs, min_size_ratio=MIN_SIZE_RATIO),
    background_remover=background_remover,
    descriptor_type=DESCRIPTOR,
    nfeatures=NFEATURES,
    rootsift=ROOTSIFT,
    orb_fast_threshold=ORB_FAST_THRESHOLD,
    orb_score_type=ORB_SCORE_TYPE,
    crop_min_ratio_single=CROP_MIN_RATIO,
    bg_crop_min_ratio=BG_CROP_MIN_RATIO,
    show_img=SHOW_VIS,
    info=INFO_PRINT,
)

# 5) Ranking + mAP
pred_orb = compute_matches(
    desc_query_orb,
    db_descs,
    gt_corresps,
    matcher=matcher,
    ratio=LOWE_RATIO,
    rate_threshold=RATE_THRESHOLD
)

map5_orb = mapk(gt_corresps, pred_orb, k=5, skip_unknown=False)
map1_orb = mapk(gt_corresps, pred_orb, k=1, skip_unknown=False)

print(f"✅ mAP@1 = {map1_orb:.4f}")
print(f"✅ mAP@5 = {map5_orb:.4f}")


✅ Loading DB descriptors from results/descriptors_db_orb.pkl
📥 Loading query images...
📥 Loading ground-truth correspondences...
→ 30 GT entries loaded


Processing queries:   3%|▎         | 1/30 [00:01<00:41,  1.45s/it]

Query 0 GT: [-1], Predicted: -1, Rate: 0.0487


Processing queries:   7%|▋         | 2/30 [00:03<00:45,  1.61s/it]

Query 1 GT: [150], Predicted: 150, Rate: 0.1440
Query 2 GT: [48, 251], Predicted: 48, Rate: 0.1422


...

Query 27 GT: [127], Predicted: 127, Rate: 0.4595
Query 28 GT: [47, 13], Predicted: 47, Rate: 0.1313


Processing queries:  97%|█████████▋| 29/30 [01:02<00:02,  2.06s/it]

Query 28 GT: [47, 13], Predicted: 13, Rate: 0.3173


Processing queries: 100%|██████████| 30/30 [01:05<00:00,  2.19s/it]

Query 29 GT: [-1], Predicted: -1, Rate: 0.0415
Min score for correct matches: 0.0125
Max score for incorrect matches: 0.056375838926174496
✅ mAP@1 = 0.8718
✅ mAP@5 = 0.8718


In [24]:
#print pred
for i in range(len(pred_orb)):
    if len(pred_orb[i]) == 2:
        print(f"Query image: {q_names[i]} - GT: {gt_corresps[i]} - Predicted DB indices left: {pred_orb[i][0][:5]}, right: {pred_orb[i][1][:5]}")
    else:
        print(f"Query image: {q_names[i]} - GT: {gt_corresps[i]} - Predicted DB indices: {pred_orb[i][:5]}")

Query image: 00000.jpg - GT: [-1] - Predicted DB indices: [-1]
Query image: 00001.jpg - GT: [150] - Predicted DB indices: (150, 95, 257, 2, 281)
Query image: 00002.jpg - GT: [48, 251] - Predicted DB indices left: (48, 164, 255, 204, 53), right: (251, 77, 47, 74, 109)
Query image: 00003.jpg - GT: [32] - Predicted DB indices: (32, 164, 143, 155, 206)
Query image: 00004.jpg - GT: [161] - Predicted DB indices: (161, 87, 204, 103, 197)
Query image: 00005.jpg - GT: [81] - Predicted DB indices: (81, 164, 17, 204, 206)
Query image: 00006.jpg - GT: [62, 38] - Predicted DB indices left: (62, 4, 164, 103, 192), right: (38, 183, 207, 47, 256)
Query image: 00007.jpg - GT: [-1] - Predicted DB indices: [-1]
Query image: 00008.jpg - GT: [128] - Predicted DB indices: [-1]
Query image: 00009.jpg - GT: [155, 258] - Predicted DB indices left: (155, 261, 7, 103, 281), right: [-1]
Query image: 00010.jpg - GT: [136, 76] - Predicted DB indices left: (136, 164, 206, 51, 267), right: (76, 166, 257, 67, 4)
Query

## CSIFT

In [ ]:
DB_FOLDER      = "../Data/BBDD/"
QUERY_FOLDER   = "../Data/Week4/qsd1_w4/"
GT_PATH        = "../Data/Week4/qsd1_w4/gt_corresps.pkl"

DESCRIPTOR         = "CSIFT"     # <- "SIFT" | "ORB" | "CSIFT"
COLOR_SPACE        = "OPPONENT"  # <- for CSIFT: "OPPONENT" | "RGB" | "HSV"

NFEATURES          = 1500        # works for SIFT/CSIFT; ORB uses same param name
ROOTSIFT           = True        # applies to SIFT/CSIFT only
ORB_FAST_THRESHOLD = 20
ORB_SCORE_TYPE     = "HARRIS"    # or "FAST"

LOWE_RATIO         = 0.76
RATE_THRESHOLD     = 0.045
MIN_SIZE_RATIO     = 0.20
CROP_MIN_RATIO     = 0.20
BG_CROP_MIN_RATIO  = 0.10
SHOW_VIS           = False
INFO_PRINT         = False

# Rate threshold = 0.125 gives:
# ✅ mAP@1 = 0.6923
# ✅ mAP@5 = 0.6923
# Rate thereshold = 0.5 gives:
# ✅ mAP@1 = 0.2564
# ✅ mAP@5 = 0.2564
# Rate thereshold = 0.045 gives:
# ✅ mAP@1 = 0.8974
# ✅ mAP@5 = 0.9060

DESC_DB_PATH   = f"results/descriptors_db_{DESCRIPTOR.lower()}_{COLOR_SPACE.lower() if DESCRIPTOR.upper()=='CSIFT' else ''}.pkl".replace("__","_")

os.makedirs("results", exist_ok=True)

# 1) Matcher
matcher = create_matcher(DESCRIPTOR)

# 2) DB descriptors
db_descs, db_names = build_or_load_db_descriptors(
    DB_FOLDER,
    DESC_DB_PATH,
    descriptor_type=DESCRIPTOR,
    nfeatures=NFEATURES,
    rootsift=ROOTSIFT,
    color_space=COLOR_SPACE,
    orb_fast_threshold=ORB_FAST_THRESHOLD,
    orb_score_type=ORB_SCORE_TYPE,
    use_cache=True,
    progress=True,
)

# 3) Queries + GT
print("📥 Loading query images...")
q_names, q_imgs = load_images_from_folder(QUERY_FOLDER)
print("📥 Loading ground-truth correspondences...")
with open(GT_PATH, "rb") as f:
    gt_corresps = pickle.load(f)
print(f"→ {len(gt_corresps)} GT entries loaded")

# 4) Query descriptors
print(f"🧠 Computing {DESCRIPTOR} for queries...")
desc_query_csift = compute_query_descriptors(
    q_imgs,
    q_names,
    preprocess_image=preprocess_image,
    segment_multiple_paintings=segment_multiple_paintings,
    validate_split_fn=lambda is_split, imgs: validate_split(is_split, imgs, min_size_ratio=MIN_SIZE_RATIO),
    background_remover=background_remover,
    descriptor_type=DESCRIPTOR,
    nfeatures=NFEATURES,
    rootsift=ROOTSIFT,
    color_space=COLOR_SPACE,
    orb_fast_threshold=ORB_FAST_THRESHOLD,
    orb_score_type=ORB_SCORE_TYPE,
    crop_min_ratio_single=CROP_MIN_RATIO,
    bg_crop_min_ratio=BG_CROP_MIN_RATIO,
    show_img=SHOW_VIS,
    info=INFO_PRINT,
)

# 5) Ranking + mAP
print("🔍 Computing matches...")
pred_csift = compute_matches(
    desc_query_csift,
    db_descs,
    gt_corresps,
    matcher=matcher,
    ratio=LOWE_RATIO,
    rate_threshold=RATE_THRESHOLD,
    descriptor_type=DESCRIPTOR,
)
map5_csift = mapk(gt_corresps, pred_csift, k=5, skip_unknown=False)
map1_csift = mapk(gt_corresps, pred_csift, k=1, skip_unknown=False)

print(f"✅ mAP@1 = {map1_csift:.4f}")
print(f"✅ mAP@5 = {map5_csift:.4f}")

✅ Loading DB descriptors from results/descriptors_db_csift_opponent.pkl
📥 Loading query images...
📥 Loading ground-truth correspondences...
→ 30 GT entries loaded
🔍 Computing matches...


Processing queries:   3%|▎         | 1/30 [00:20<09:52, 20.42s/it]

Query 0 GT: [-1], Predicted: 137, Rate: 0.0687


Processing queries:   7%|▋         | 2/30 [00:39<09:16, 19.89s/it]

Query 1 GT: [150], Predicted: 150, Rate: 0.0827
Query 2 GT: [48, 251], Predicted: 48, Rate: 0.1490


...

Query 27 GT: [127], Predicted: 127, Rate: 0.0567
Query 28 GT: [47, 13], Predicted: 189, Rate: 0.0477


Processing queries:  97%|█████████▋| 29/30 [10:59<00:25, 25.45s/it]

Query 28 GT: [47, 13], Predicted: 13, Rate: 0.1204


Processing queries: 100%|██████████| 30/30 [11:19<00:00, 22.66s/it]

Query 29 GT: [-1], Predicted: -1, Rate: 0.0273
Min score for correct matches: 0.0013324450366422385
Max score for incorrect matches: 0.06866666666666667
✅ mAP@1 = 0.8974
✅ mAP@5 = 0.9060


In [25]:
#print pred
for i in range(len(pred_csift)):
    if len(pred_csift[i]) == 2:
        print(f"Query image: {q_names[i]} - GT: {gt_corresps[i]} - Predicted DB indices left: {pred_csift[i][0][:5]}, right: {pred_csift[i][1][:5]}")
    else:
        print(f"Query image: {q_names[i]} - GT: {gt_corresps[i]} - Predicted DB indices: {pred_csift[i][:5]}")

Query image: 00000.jpg - GT: [-1] - Predicted DB indices: (137, 209, 136, 106, 248)
Query image: 00001.jpg - GT: [150] - Predicted DB indices: (150, 268, 95, 178, 86)
Query image: 00002.jpg - GT: [48, 251] - Predicted DB indices left: (48, 8, 11, 62, 150), right: (251, 54, 226, 44, 134)
Query image: 00003.jpg - GT: [32] - Predicted DB indices: (32, 189, 8, 88, 109)
Query image: 00004.jpg - GT: [161] - Predicted DB indices: (161, 192, 44, 88, 106)
Query image: 00005.jpg - GT: [81] - Predicted DB indices: (81, 133, 88, 137, 31)
Query image: 00006.jpg - GT: [62, 38] - Predicted DB indices left: (62, 136, 140, 141, 171), right: (38, 268, 281, 88, 117)
Query image: 00007.jpg - GT: [-1] - Predicted DB indices: [-1]
Query image: 00008.jpg - GT: [128] - Predicted DB indices: [-1]
Query image: 00009.jpg - GT: [155, 258] - Predicted DB indices left: (155, 282, 131, 107, 157), right: (258, 160, 114, 151, 285)
Query image: 00010.jpg - GT: [136, 76] - Predicted DB indices left: (136, 164, 214, 165,